# Loading an "External System" Transaction XML into LUSID

The purpose of this notebook  is to demonstrate how a user could load a transaction XML file from another "External System" into LUSID. The notebook will walk through the following steps: <br>

(1) Load transaction XML into DataFrame. <br>
(2) Scan for portfolio code in DataFrame. If portfolio code from DataFrame does not exist in LUSID, create the portfolio in LUSID. <br>
(3) Translate the transaction data in DataFrame into data types which LUSID will recognise (e.g. transform "strings" to "datetimes"). <br>
(4) Use client identifiers to create placeholder LUIDS (these can be enriched at a later date once we receive the "External System" instrument master file). <br>
(5) Load the transaction into LUSID using the Transactions API. <br><br>


## Setup: Load libraries and create our LUSID API client


In [9]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import lusidtools.cocoon.cocoon as cocoon_tools
from lusidjam import RefreshingToken

# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
import xml.etree.ElementTree as ET
import io

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Client Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.5.3388.0


As part of the setup, we need to define a Scope. Scopes in LUSID are used to partition data. In this example, we will use the Scope "EXT_SYSTEM" to represent an external trading system. 

In [10]:
scope = "EXT_SYSTEM"

## Step 1: Scrape Transaction XML data into a Python DataFrame

First we need to load the XML file into a Python DataFrame. This will enable us to work with the data, then pass it to the relevant APIs.

In [11]:
# Load the XML file into memory

ext_system_file = os.path.join("data", "ext_system_file.xml")

In [12]:
# Create a Python object from the XML file
# List out the individual tags which will need to be mapped

tree = ET.parse(ext_system_file)
root = tree.getroot()
list_tags = list(set([elem.tag for elem in root.iter()]))
for tag in list_tags:
    print(tag)

TRD_TRADER
INST_TICKER
INST_DESC
PORT_NAME
CLIENT_ID
TRD_CURRENCY
TRD_PRINCIPAL
TRANSACTIONS
TRD_TRADE_DATE
TRD_TRANS_TYPE
TRD_COUNTERPARTY
TRD_PRICE
TRD_TOUCH_COUNT
TRD_QUANTITY
TRD_NUMBER
TRD_LOCATION
INST_GROUP
TRADE
INST_TYPE
TRD_SETTLE_DATE


Create a list of the tags which we want to load into LUSID. For the purposes of this example, we will select a subset of fields from the sample XML file. In production, you may need to load more.

In [13]:
list_tags = [
    
    "TRD_TRANS_TYPE",
    "TRD_QUANTITY",
    "TRD_TRADE_DATE",
    "PORT_NAME",
    "INST_GROUP",
    "TRD_CURRENCY",
    "TRD_TOUCH_COUNT",
    "INST_DESC",
    "INST_TYPE",
    "TRD_PRINCIPAL",
    "TRD_TRADER",
    "TRD_PRICE",
    "TRD_COUNTERPARTY",
    "INST_TICKER",
    "CLIENT_ID",
    "TRD_SETTLE_DATE",
    "TRD_LOCATION",
    "TRD_NUMBER"
    
]

Iterate over the "External System" transaction XML and scrape the results into a Pandas DataFrame. We scrape the results into a DataFrame for two reasons:<br>

(1) We can use the DataFrame methods to transform the data into a format LUSID likes. <br>
(2) We can then use the SDKs to pass the DataFrame directly to LUSID<br>
<br>

In [15]:
# Define an empty list which will be used to create a DataFrame from XML results 
rows= []

# Iterate over the XML to find transactions in the sub-XML nested under the "TRADE" tag.
# Store each individual transaction as a dictionary.
# Then add each dictionary (representing a transaction) to a list.
# Use this list of dictionaries to create a DataFrame.

for trade in root.iter('TRADE'):
    
    trade_list = []
    for tag in list_tags:
        
        try:       
            trade_list.append((trade.find(tag).text))
        except AttributeError:
            trade_list.append(None)

    rows.append({"TRD_TRANS_TYPE": trade_list[0], 
                 "TRD_QUANTITY": trade_list[1], 
                 "TRD_TRADE_DATE": trade_list[2],
                 "PORT_NAME": trade_list[3],
                 "INST_GROUP": trade_list[4],
                 "TRD_CURRENCY": trade_list[5],
                 "TRD_TOUCH_COUNT": trade_list[6],
                 "INST_DESC": trade_list[7],
                 "INST_TYPE": trade_list[8],
                 "TRD_PRINCIPAL": trade_list[9],
                 "TRD_TRADER": trade_list[10],
                 "TRD_PRICE": trade_list[11],
                 "TRD_COUNTERPARTY": trade_list[12],
                 "INST_TICKER": trade_list[13],
                 "CLIENT_ID": trade_list[14],
                 "TRD_SETTLE_DATE": trade_list[15],
                 "TRD_LOCATION": trade_list[16],
                 "TRD_NUMBER": trade_list[17]
                
                })
    
ext_system_df = pd.DataFrame(rows)

Let's take a look at the DataFrame. The results look good. We see trade details - broker, security type, price etc. 

In [16]:
ext_system_df

,CLIENT_ID,INST_DESC,INST_GROUP,INST_TICKER,INST_TYPE,PORT_NAME,TRD_COUNTERPARTY,TRD_CURRENCY,TRD_LOCATION,TRD_NUMBER,TRD_PRICE,TRD_PRINCIPAL,TRD_QUANTITY,TRD_SETTLE_DATE,TRD_TOUCH_COUNT,TRD_TRADER,TRD_TRADE_DATE,TRD_TRANS_TYPE
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP,EQUITY,IBM,COMMON_STOCK,TEST_PORT1,GS,USD,LONDON,0011223,142.99,7864450.000,55000.0000000000,8/20/2018,2,TEST_TRADER,8/16/2018,BUY
1,EXT1YU6H6,BANK OF AMERICA CORP,EQUITY,BAC,COMMON_STOCK,TEST_PORT1,JPM,USD,LONDON,0011224,28.99,2899000.000,100000.0000000000,9/20/2018,1,TEST_TRADER,9/16/2018,BUY
2,EXT1YU6H7,WALMART INC,EQUITY,WMT,COMMON_STOCK,TEST_PORT2,JPM,USD,LONDON,0011225,116.99,11699000,100000.0000000000,10/20/2018,1,TEST_TRADER,10/16/2018,SELL
3,EXT1YU6H8,3M CO,EQUITY,MMM,COMMON_STOCK,TEST_PORT2,MSUSA,USD,LONDON,0011226,155.99,11599000,100000.0000000000,5/20/2018,6,TEST_TRADER,5/16/2018,BUY


## Step 2: Create portfolios in LUSID

Next we need to create portfolios in LUSID. The "External System" transaction XMLs will be loaded into these portfolio.

In [8]:
# Create a list of the unique portfolios from the transaction XML file

portfolios = list(set(ext_system_df["PORT_NAME"].tolist()))

In [9]:
# Prepare a DataFrame of portfolios

rows = []
for port in portfolios:
    
    rows.append(
        
        {'FundCode': port,
         'display_name': 'External System Positions {}'.format(port),
         'created': '2016-10-09',
         'base_currency': 'USD',
        'description': 'A portfolio to hold trades from an external system - {}'.format(port)}
        
    )
    

portfolios_data = pd.DataFrame(rows)
portfolios_data

,FundCode,base_currency,created,description,display_name
0,TEST_PORT2,USD,2016-10-09,A portfolio to hold trades from an external sy...,External System Positions TEST_PORT2
1,TEST_PORT1,USD,2016-10-09,A portfolio to hold trades from an external sy...,External System Positions TEST_PORT1


In [10]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'created': 'created',
  'description': 'description',
  'accounting_method': None
}

In [11]:
# Create the portfolios if they do not already exist in LUSID

responses = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=portfolios_data, 
    mapping_required=portfolio_mapping_required, 
    mapping_optional=portfolio_mapping_optional,
    file_type='portfolio')

for response in responses["portfolios"]["success"]:
    prettyprint.portfolio_response(response)

## Step 3: Translate the data into a LUSID-friendly format

The script above loads "Trade Date" and "Settle Date" as strings. LUSID needs these as datetimes.

In [12]:
ext_system_df['TRD_TRADE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_TRADE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

ext_system_df['TRD_SETTLE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_SETTLE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

In [13]:
ext_system_df

,CLIENT_ID,INST_DESC,INST_GROUP,INST_TICKER,INST_TYPE,PORT_NAME,TRD_COUNTERPARTY,TRD_CURRENCY,TRD_LOCATION,TRD_NUMBER,TRD_PRICE,TRD_PRINCIPAL,TRD_QUANTITY,TRD_SETTLE_DATE,TRD_TOUCH_COUNT,TRD_TRADER,TRD_TRADE_DATE,TRD_TRANS_TYPE
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP,EQUITY,IBM,COMMON_STOCK,TEST_PORT1,GS,USD,LONDON,0011223,142.99,7864450.000,55000.0000000000,2018-08-20 00:00:00+00:00,2,TEST_TRADER,2018-08-16 00:00:00+00:00,BUY
1,EXT1YU6H6,BANK OF AMERICA CORP,EQUITY,BAC,COMMON_STOCK,TEST_PORT1,JPM,USD,LONDON,0011224,28.99,2899000.000,100000.0000000000,2018-09-20 00:00:00+00:00,1,TEST_TRADER,2018-09-16 00:00:00+00:00,BUY
2,EXT1YU6H7,WALMART INC,EQUITY,WMT,COMMON_STOCK,TEST_PORT2,JPM,USD,LONDON,0011225,116.99,11699000,100000.0000000000,2018-10-20 00:00:00+00:00,1,TEST_TRADER,2018-10-16 00:00:00+00:00,SELL
3,EXT1YU6H8,3M CO,EQUITY,MMM,COMMON_STOCK,TEST_PORT2,MSUSA,USD,LONDON,0011226,155.99,11599000,100000.0000000000,2018-05-20 00:00:00+00:00,6,TEST_TRADER,2018-05-16 00:00:00+00:00,BUY


## Step 4: Create placeholder LUIDs from "External System" client IDs

In an ideal world, we would setup our instrument master *before* upserting transactions. Unfortunately, that is not always possible. In this example, the external system or party has sent us the transaction file before an instrument master file. Therefore, as a workaround, we will map the unique CLIENT_IDs to a placeholder LUSID Unique Identifer (called a LUID). These LUIDs can be updated with more data (Public IDs, Sectors, Ratings etc) when the client "Instrument Master" file is sent to us later in the day.<br>

In [14]:
# Create a DataFrame containing the "External System" ID

instruments = ext_system_df[["CLIENT_ID", "INST_DESC"]]
instruments

,CLIENT_ID,INST_DESC
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP
1,EXT1YU6H6,BANK OF AMERICA CORP
2,EXT1YU6H7,WALMART INC
3,EXT1YU6H8,3M CO


In [15]:
instrument_mapping_required = {
  'name': 'INST_DESC'
}

instrument_identifier_mapping = {
    'ClientInternal': 'CLIENT_ID'
}

instrument_mapping_optional = {}

In [16]:
response = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    data_frame=instruments, 
    identifier_mapping=instrument_identifier_mapping, 
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type='instruments',
    scope='ExtSysProperties001')

prettyprint.instrument_response(response["instruments"]["success"][0])

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,WALMART INC,EXT1YU6H7,LUID_1IQX5N1J
1,BANK OF AMERICA CORP,EXT1YU6H6,LUID_RE6ECJE8
2,INTERNATIONAL BUSINESS MACHINES CORP,EXT1YU6H5,LUID_SPJFM4L1
3,3M CO,EXT1YU6H8,LUID_D0CJRFUB


## Step 5: Load transactions into LUSID

Finally, we want to load the transactions into LUSID. 

In [19]:
transaction_field_mapping_required = {
    "code": "PORT_NAME",
    "transaction_id": "TRD_NUMBER",
    "type": 'TRD_TRANS_TYPE',
    "transaction_date": 'TRD_TRADE_DATE',
    "settlement_date": 'TRD_SETTLE_DATE',
    "units": "TRD_QUANTITY",
    "transaction_price.price": "TRD_PRICE",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "TRD_PRINCIPAL",
    "total_consideration.currency": "TRD_CURRENCY",
    "transaction_currency": "TRD_CURRENCY"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "$1",
    "source": f"${scope}"
}

transaction_identifier_mapping = {
    'ClientInternal': "CLIENT_ID"
}

Up to row 0


In [21]:
responses = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=ext_system_df,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction")

for response in responses["transactions"]["success"]:
    print ('\n')
    prettyprint.transactions_response(response, scope, response.href.split("/")[7])

data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing transaction properties complete


Transactions Successfully Upserted into Portfolio
Scope: EXT_SYSTEM
Code: portfolio
Transactions Effective From: 2018-10-16 00:00:00+00:00
Transactions Created On: 2019-10-10 16:25:27.299610+00:00



The transactions are now visible in the front-end LUSID tool.

![Init](img/read_transactions.png)